In [58]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [61]:
def mat_factorization(mat, user_features, item_features, features, epoch=100, alpha=0.002, beta=0.02):
    for val in tqdm(range(epoch)):
      for value in range(len(mat)):
          for values in range(len(mat[value])):
              if mat[value][values] > 0:
                  error_mat = mat[value][values] - np.dot(user_features[value,:], item_features[:, values]) # To find the error between the original given matrix and the matrices obtained from the rand function.

                  for data in range(features):
                      user_features[value][data] = user_features[value][data] + alpha * (2 * item_features[data][values] * error_mat - beta * user_features[value][data]) # Using the gradient descent function to reduce the error between matrices
                      item_features[data][values] = item_features[data][values] + alpha * (2 * user_features[value][data] * error_mat - beta * item_features[data][values])

    return user_features, item_features.T

In [34]:
rating = pd.read_csv("ratings.csv")
link = pd.read_csv("links.csv")
g_tags = pd.read_csv("genome-tags.csv")
g_scores = pd.read_csv("genome-scores.csv")
tag = pd.read_csv("tags.csv")
movie = pd.read_csv("movies.csv")

In [35]:
print(rating)
rating.shape

          userId  movieId  rating   timestamp
0              1        2     3.5  1112486027
1              1       29     3.5  1112484676
2              1       32     3.5  1112484819
3              1       47     3.5  1112484727
4              1       50     3.5  1112484580
...          ...      ...     ...         ...
20000258  138493    68954     4.5  1258126920
20000259  138493    69526     4.5  1259865108
20000260  138493    69644     3.0  1260209457
20000261  138493    70286     5.0  1258126944
20000262  138493    71619     2.5  1255811136

[20000263 rows x 4 columns]


(20000263, 4)

In [15]:
movie

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [36]:
movie_rating = pd.merge(rating, movie, on='movieId')
movie_rating

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,1112486027,Jumanji (1995),Adventure|Children|Fantasy
1,5,2,3.0,851527569,Jumanji (1995),Adventure|Children|Fantasy
2,13,2,3.0,849082742,Jumanji (1995),Adventure|Children|Fantasy
3,29,2,3.0,835562174,Jumanji (1995),Adventure|Children|Fantasy
4,34,2,3.0,846509384,Jumanji (1995),Adventure|Children|Fantasy
...,...,...,...,...,...,...
20000258,138301,121017,3.5,1420558479,The Gentleman from Epsom (1962),Comedy|Crime
20000259,138301,121019,4.5,1420558606,The Great Spy Chase (1964),Action|Comedy|Thriller
20000260,138301,121021,4.5,1420558687,Taxi for Tobruk (1961),Drama|War
20000261,138406,110167,4.5,1396184127,"Judge and the Assassin, The (Juge et l'assassi...",Crime|Drama


In [37]:
movie_rating['userId'].unique()

array([     1,      5,     13, ...,  40432,    861, 121610])

In [38]:
movie_rating = movie_rating.iloc[:1000000]

In [39]:
movie_rating

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,1112486027,Jumanji (1995),Adventure|Children|Fantasy
1,5,2,3.0,851527569,Jumanji (1995),Adventure|Children|Fantasy
2,13,2,3.0,849082742,Jumanji (1995),Adventure|Children|Fantasy
3,29,2,3.0,835562174,Jumanji (1995),Adventure|Children|Fantasy
4,34,2,3.0,846509384,Jumanji (1995),Adventure|Children|Fantasy
...,...,...,...,...,...,...
999995,90990,1198,3.5,1388278593,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure
999996,90991,1198,5.0,1007505047,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure
999997,90995,1198,5.0,947217797,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure
999998,90997,1198,3.0,956715455,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure


In [40]:
movie_rating['title'].unique()

array(['Jumanji (1995)',
       'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
       'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
       'Seven (a.k.a. Se7en) (1995)', 'Usual Suspects, The (1995)',
       'Rumble in the Bronx (Hont faan kui) (1995)', 'Rob Roy (1995)',
       'Clerks (1994)',
       'Interview with the Vampire: The Vampire Chronicles (1994)',
       'Star Wars: Episode IV - A New Hope (1977)',
       'Léon: The Professional (a.k.a. The Professional) (Léon) (1994)',
       'Pulp Fiction (1994)', 'Shawshank Redemption, The (1994)',
       "What's Eating Gilbert Grape (1993)", 'Mask, The (1994)',
       'Blade Runner (1982)', 'Terminator 2: Judgment Day (1991)',
       'Silence of the Lambs, The (1991)', 'Dragonheart (1996)',
       'Wizard of Oz, The (1939)', '2001: A Space Odyssey (1968)',
       'Escape to Witch Mountain (1975)', 'Die Hard (1988)',
       'Fish Called Wanda, A (1988)',
       "Monty Python's Life of Brian (1979)", 'Reservoir Dogs (1

In [41]:
matrix = movie_rating.pivot_table(values='rating', index='userId', columns='title').fillna(0.0)
matrix

title,2001: A Space Odyssey (1968),Blade Runner (1982),"City of Lost Children, The (Cité des enfants perdus, La) (1995)",Clerks (1994),Die Hard (1988),Dragonheart (1996),E.T. the Extra-Terrestrial (1982),Escape to Witch Mountain (1975),"Fish Called Wanda, A (1988)",Interview with the Vampire: The Vampire Chronicles (1994),Jumanji (1995),Léon: The Professional (a.k.a. The Professional) (Léon) (1994),"Mask, The (1994)",Monty Python and the Holy Grail (1975),Monty Python's Life of Brian (1979),One Flew Over the Cuckoo's Nest (1975),Platoon (1986),Pulp Fiction (1994),Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),Reservoir Dogs (1992),Rob Roy (1995),Rumble in the Bronx (Hont faan kui) (1995),Seven (a.k.a. Se7en) (1995),"Shawshank Redemption, The (1994)","Silence of the Lambs, The (1991)",Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back (1980),Terminator 2: Judgment Day (1991),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"Usual Suspects, The (1995)",What's Eating Gilbert Grape (1993),"Wizard of Oz, The (1939)"
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.5,4.0,3.5,4.0,4.0,3.0,4.0,3.5,4.0,4.0,3.5,4.0,3.5,3.5,3.5,3.5,4.0,4.0,4.5,3.5,4.0,3.5,3.5,4.0,3.5,4.0,4.5,3.5,3.5,3.5,3.5,3.5
2,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0
3,5.0,5.0,0.0,5.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0,3.0,4.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,5.0,0.0,5.0,0.0,5.0,0.0,3.0,0.0,3.0,5.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,3.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138489,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0
138490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,3.0,0.0


In [42]:
matrix = matrix.values
matrix

array([[3.5, 4. , 3.5, ..., 3.5, 3.5, 3.5],
       [5. , 5. , 0. , ..., 0. , 0. , 0. ],
       [5. , 5. , 0. , ..., 5. , 3. , 4. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 3. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 4. , 0. ]])

In [62]:
features = 3
user_features = np.random.rand(len(matrix), features) # Initializing the user and item matrix using the np.random.rand function.
item_features = np.random.rand(features, len(matrix[0]))
user_mat, item_mat = mat_factorization(matrix, user_features, item_features, features)

100%|██████████| 100/100 [45:02<00:00, 27.02s/it]


In [63]:
final_mat = np.dot(user_mat, item_mat.T)
final_mat

array([[3.58031766, 3.74455408, 3.77347425, ..., 3.85962192, 3.46083621,
        4.01243069],
       [4.88608066, 4.98109972, 4.68060876, ..., 5.04041946, 4.28004941,
        4.68810933],
       [4.42670019, 4.53580966, 4.42483889, ..., 4.86922357, 4.38058795,
        4.40331108],
       ...,
       [4.01942315, 4.15796389, 4.13720249, ..., 4.42823643, 4.01219493,
        4.23528818],
       [4.22772403, 4.34484995, 4.14656135, ..., 4.34389882, 3.69589721,
        4.26474347],
       [4.18405621, 4.39586655, 4.54101472, ..., 4.69913866, 4.36119241,
        4.81940349]])

In [64]:
pd.DataFrame(user_mat).to_csv('user_mat.csv')
pd.DataFrame(item_mat).to_csv('item_mat.csv')
pd.DataFrame(final_mat).to_csv('final_mat.csv')

In [66]:
final_mat.shape

(125917, 32)

In [70]:
final_values = pd.DataFrame(final_mat, index=movie_rating['userId'].unique(), columns=movie_rating['title'].unique())
final_values

,Jumanji (1995),"City of Lost Children, The (Cité des enfants perdus, La) (1995)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Seven (a.k.a. Se7en) (1995),"Usual Suspects, The (1995)",Rumble in the Bronx (Hont faan kui) (1995),Rob Roy (1995),Clerks (1994),Interview with the Vampire: The Vampire Chronicles (1994),Star Wars: Episode IV - A New Hope (1977),Léon: The Professional (a.k.a. The Professional) (Léon) (1994),Pulp Fiction (1994),"Shawshank Redemption, The (1994)",What's Eating Gilbert Grape (1993),"Mask, The (1994)",Blade Runner (1982),Terminator 2: Judgment Day (1991),"Silence of the Lambs, The (1991)",Dragonheart (1996),"Wizard of Oz, The (1939)",2001: A Space Odyssey (1968),Escape to Witch Mountain (1975),Die Hard (1988),"Fish Called Wanda, A (1988)",Monty Python's Life of Brian (1979),Reservoir Dogs (1992),Platoon (1986),E.T. the Extra-Terrestrial (1982),Monty Python and the Holy Grail (1975),One Flew Over the Cuckoo's Nest (1975),Star Wars: Episode V - The Empire Strikes Back (1980),Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
1,3.580318,3.744554,3.773474,3.228030,3.853068,3.410272,3.777429,3.143986,3.517883,3.391464,3.341456,3.697515,3.296243,3.849853,3.639293,3.846417,3.629677,3.623285,4.036193,3.429616,3.488236,3.356698,3.634656,4.200695,3.843715,4.067150,4.004640,3.884122,3.596229,3.859622,3.460836,4.012431
5,4.886081,4.981100,4.680609,4.334408,4.760520,3.982541,4.354573,3.489686,4.452647,4.180842,3.745392,4.774261,3.813440,5.006380,4.820810,4.977397,4.651669,4.993159,5.087088,4.686165,4.053612,4.094010,4.762842,5.295082,4.923827,5.084659,5.033571,4.854494,4.667768,5.040419,4.280049,4.688109
13,4.426700,4.535810,4.424839,4.532705,4.129600,2.987588,4.092947,3.356720,4.166196,3.972931,3.197545,4.575093,3.102332,4.695819,4.486566,4.773138,4.344183,5.015106,4.340455,4.685041,4.147836,3.714992,4.659747,4.962622,4.707247,4.517861,4.468405,4.236496,4.517419,4.869224,4.380588,4.403311
29,2.761770,2.835773,2.715201,2.436571,2.780292,2.387208,2.586963,2.096214,2.568619,2.429132,2.259172,2.732963,2.282769,2.863882,2.743943,2.847215,2.673564,2.794009,2.955779,2.629659,2.387890,2.393944,2.710344,3.058129,2.825862,2.954333,2.919956,2.824019,2.665944,2.873820,2.471324,2.773039
34,4.612947,4.748769,4.696576,4.766209,4.374575,3.190085,4.402565,3.637877,4.404767,4.222203,3.454046,4.824542,3.325687,4.941792,4.705110,5.031656,4.582591,5.243646,4.577480,4.903932,4.461497,3.950535,4.905611,5.250911,4.970182,4.778873,4.721625,4.478728,4.761303,5.126503,4.662696,4.724331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90326,2.751317,2.919898,3.070453,2.602669,3.078951,2.717706,3.166335,2.687574,2.829293,2.770603,2.778335,2.964082,2.678735,3.058998,2.859962,3.080862,2.900948,2.851727,3.182635,2.706993,2.970695,2.725782,2.909406,3.386218,3.090783,3.249674,3.190897,3.091039,2.883241,3.084072,2.875490,3.344458
90543,3.780333,3.972980,4.179396,4.156996,3.803899,2.779730,4.113981,3.504030,3.853492,3.778701,3.225211,4.194962,2.991749,4.246126,3.980047,4.369608,3.972026,4.440668,3.897065,4.168446,4.223034,3.516058,4.252220,4.609237,4.341520,4.149976,4.082524,3.867521,4.138618,4.435343,4.247684,4.373232
90763,4.019423,4.157964,4.137202,3.996353,3.966509,3.093944,3.959890,3.280293,3.872680,3.719159,3.233463,4.189403,3.139133,4.311881,4.096229,4.365923,4.018672,4.420027,4.151488,4.147535,3.899253,3.541274,4.217803,4.619060,4.327583,4.285430,4.229367,4.040439,4.116766,4.428236,4.012195,4.235288
90766,4.227724,4.344850,4.146561,3.591552,4.335328,3.849626,3.982760,3.220338,3.925814,3.706888,3.563342,4.144059,3.627268,4.363936,4.182745,4.315644,4.083670,4.157222,4.618577,3.921006,3.593705,3.698309,4.081584,4.673733,4.290657,4.577047,4.522711,4.393400,4.029992,4.343899,3.695897,4.264743


In [77]:
user_id = int(input("Enter the user Id to find the predictions"))
final_values.iloc[user_id]

Enter the user Id to find the predictions55


Jumanji (1995)                                                                    3.771200
City of Lost Children, The (Cité des enfants perdus, La) (1995)                   3.859477
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)                                         3.621456
Seven (a.k.a. Se7en) (1995)                                                       3.085860
Usual Suspects, The (1995)                                                        3.854713
Rumble in the Bronx (Hont faan kui) (1995)                                        3.485666
Rob Roy (1995)                                                                    3.451320
Clerks (1994)                                                                     2.762491
Interview with the Vampire: The Vampire Chronicles (1994)                         3.445323
Star Wars: Episode IV - A New Hope (1977)                                         3.231099
Léon: The Professional (a.k.a. The Professional) (Léon) (1994)                    3.138976